In [15]:
import pandas as pd
import os
import sys
from embeds import fix_corrupt, multi_inner_align
import numpy as np
from hypertools.tools import align
import torch
from tqdm.notebook import tqdm
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

sys.path.append('..')
from rca.rca import run_rca

In [21]:
def read_txt(f):
    pulled = {}
    for line in f:
        word, *vec = line.split()
        pulled[word] = vec
    pulled = fix_corrupt(pulled)
    return pd.DataFrame(pulled).T.astype(float)

def standardize(df):
    # Standardize
    df = (df - df.mean()) / df.std()
    # Set Nans to zero
    df = df.fillna(0.0)
    return df


def read_fmri(path: str) -> pd.DataFrame:
    dfs = []

    for f_name in os.listdir(path):

        if f_name.endswith('.txt'):
            with open(path + f_name, 'r') as f:
                dfs.append(read_txt(f))

        elif f_name.endswith('pth'):
            fmri_dict = torch.load(path + f_name, weights_only=False)
            voc, vecs = fmri_dict['dico'], fmri_dict['vectors'].numpy()
            dfs.append(pd.DataFrame(vecs, index=voc, dtype=float))

        else:
            pass

    # Align indices
    dfs = list(multi_inner_align(dfs))

    # Standardize
    dfs = [standardize(df) for df in dfs]

    return dfs

fmris_text_cognival = read_fmri('../../data/fmri_text_cognival/')
fmris_speech_cognival = read_fmri('../../data/fmri_speech_cognival/')

fmris_text_denoise_128d = read_fmri('../../data/fmri_text_denoise/128d/')
fmris_text_denoise_256d = read_fmri('../../data/fmri_text_denoise/256d/')
fmris_text_denoise_512d = read_fmri('../../data/fmri_text_denoise/512d/')

print(
    len(fmris_text_cognival), len(fmris_speech_cognival), len(fmris_text_denoise_128d),
    len(fmris_text_denoise_256d), len(fmris_text_denoise_512d)
)

8 27 8 8 8


In [23]:
# --- Hyper aligning individuals ---
def hyper_align(dfs: list) -> pd.DataFrame:
    df = np.mean(align(dfs, align='hyper'), axis=0)
    return pd.DataFrame(df, index=dfs[0].index)

# cognival
fMRI_text_cognival = hyper_align(fmris_text_cognival)
fMRI_speech_cognival = hyper_align(fmris_speech_cognival)

# denoise
fMRI_text_denoise_128d = hyper_align(fmris_text_denoise_128d)
fMRI_text_denoise_256d = hyper_align(fmris_text_denoise_256d)
fMRI_text_denoise_512d = hyper_align(fmris_text_denoise_512d)

## Model comparison

In [24]:
i = 1
embeds = {
    'fMRI_text_cognival_participant': fmris_text_cognival[i],
    'fMRI_text_cognival': fMRI_text_cognival,

    'fMRI_text_denoise_128d_participant': fmris_text_denoise_128d[i],
    'fMRI_text_denoise_256d_participant': fmris_text_denoise_256d[i],
    'fMRI_text_denoise_512d_participant': fmris_text_denoise_512d[i],

    'fMRI_text_denoise_128d': fMRI_text_denoise_128d,
    'fMRI_text_denoise_256d': fMRI_text_denoise_256d,
    'fMRI_text_denoise_512d': fMRI_text_denoise_512d
}

# Aligning for controlled comparison
embeds = dict(zip(embeds.keys(), multi_inner_align(embeds.values())))

# Standardizing on same vocabulary
embeds = {name: standardize(embed) for name, embed in embeds.items()}


{name: embed.shape for name, embed in embeds.items()}

{'fMRI_text_cognival_participant': (1153, 1000),
 'fMRI_text_cognival': (1153, 1000),
 'fMRI_text_denoise_128d_participant': (1153, 128),
 'fMRI_text_denoise_256d_participant': (1153, 256),
 'fMRI_text_denoise_512d_participant': (1153, 512),
 'fMRI_text_denoise_128d': (1153, 128),
 'fMRI_text_denoise_256d': (1153, 256),
 'fMRI_text_denoise_512d': (1153, 512)}

In [25]:
norms = pd.read_csv('../../data/psychNorms/psychNorms_processed.zip', index_col=0, low_memory=False, compression='zip')
norms_meta = pd.read_csv('../../data/psychNorms/psychNorms_metadata_processed.csv', index_col='norm')
norms

,frequency_lund,frequency_kucera,frequency_subtlexus,frequency_subtlexuk,frequency_blog_gimenes,frequency_twitter_gimenes,frequency_news_gimenes,frequency_written_cobuild,frequency_spoken_cobuild,context_diversity_subtlexus,...,person_vanarsdall,goals_vanarsdall,movement_vanarsdall,concreteness_vanarsdall,familiarity_vanarsdall,imageability_vanarsdall,familiarity_fear,aoa_fear,imageability_fear,sensory_experience_juhasz2013
word,,,,,,,,,,,,,,,,,,,,,
'em,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.3617,1.9138,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'neath,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
're,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.9031,1.6335,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'shun,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'tis,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.4771,0.6021,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
shrick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.62,4.38,2.93,NaN
post office,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.79,3.07,5.29,NaN
fishing rod,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.29,3.38,5.64,NaN


In [26]:
# Testing only on numeric norms
norms_to_test = []
for norm_name in norms.columns:
    dtype = norms_meta.loc[norm_name, 'type']
    n = norms[norm_name].count()
    if dtype == 'numeric' and n >= 3000:
        norms_to_test.append(norm_name)

norms = norms[norms_to_test]
norms

,frequency_lund,frequency_kucera,frequency_subtlexus,frequency_subtlexuk,frequency_blog_gimenes,frequency_twitter_gimenes,frequency_news_gimenes,frequency_written_cobuild,frequency_spoken_cobuild,context_diversity_subtlexus,...,aoa_cortese,aoa_rt_cortese,imageability_rt_cortese,imageability_cortese,perc_known_winter,iconicity_winter2023,aoa_schock,aoa_rt_schock,iconicity_winter2017,sensory_experience_juhasz2013
word,,,,,,,,,,,,,,,,,,,,,
'em,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.3617,1.9138,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'neath,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
're,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.9031,1.6335,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'shun,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'tis,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.4771,0.6021,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
shrick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
post office,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fishing rod,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
results = run_rca(embeds, norms, norms_meta)
results

  0%|          | 0/8 [00:00<?, ?it/s]

fMRI_text_cognival_participant:   0%|          | 0/88 [00:00<?, ?it/s]

                                  norm  train_n   r2_mean     r2_sd check
56         n_semantic_neighbors_shaoul      881  0.018786  0.037550  pass
29                    haptic_lancaster      712  0.007802  0.014477  pass
57  distance_semantic_neighbors_shaoul      881  0.005233  0.035730  pass
45                    valence_mohammad      501  0.003122  0.016986  pass
36                  hand_arm_lancaster      712 -0.001156  0.024811  pass
50                    arousal_mohammad      501 -0.002012  0.016566  pass
53                  dominance_mohammad      501 -0.004229  0.026664  pass
32             interoceptive_lancaster      712 -0.004505  0.010642  pass
16                prevalence_brysbaert      718 -0.005420  0.009512  pass
74              naming_accuracy_balota      882 -0.005446  0.009649  pass


fMRI_text_cognival:   0%|          | 0/88 [00:00<?, ?it/s]

                                 norm  train_n   r2_mean     r2_sd check
2                 frequency_subtlexus      877  0.088550  0.109966  pass
3                 frequency_subtlexuk      880  0.081935  0.100987  pass
0                      frequency_lund      905  0.079998  0.099975  pass
13     context_diversity_news_gimenes      884  0.078482  0.104070  pass
6              frequency_news_gimenes      884  0.078479  0.105064  pass
12  context_diversity_twitter_gimenes      884  0.076833  0.102312  pass
1                    frequency_kucera      872  0.076807  0.136276  pass
5           frequency_twitter_gimenes      884  0.076595  0.102471  pass
11     context_diversity_blog_gimenes      884  0.076045  0.109708  pass
4              frequency_blog_gimenes      884  0.075911  0.112028  pass


...

fMRI_text_denoise_256d:   0%|          | 0/88 [00:00<?, ?it/s]

                                   norm  train_n   r2_mean     r2_sd check
28                   auditory_lancaster      712  0.060283  0.025268  pass
23                 imageability_glasgow      303  0.027633  0.072153  pass
77  recognition_memory_accuracy_cortese      292  0.022207  0.042576  pass
21               concreteness_brysbaert      712  0.001204  0.016222  pass
32              interoceptive_lancaster      712  0.000119  0.015421  pass
82                    perc_known_winter      599 -0.000346  0.002209  pass
50                     arousal_mohammad      501 -0.000626  0.009732  pass
29                     haptic_lancaster      712 -0.001221  0.010973  pass
57   distance_semantic_neighbors_shaoul      881 -0.001361  0.009737  pass
68     visual_lexical_accuracy_keuleers      793 -0.002046  0.002284  pass


fMRI_text_denoise_512d:   0%|          | 0/88 [00:00<?, ?it/s]

                                  norm  train_n   r2_mean     r2_sd check
28                  auditory_lancaster      712  0.047891  0.009600  pass
23                imageability_glasgow      303  0.034378  0.064394  pass
22                concreteness_glasgow      303  0.031313  0.051942  pass
56         n_semantic_neighbors_shaoul      881  0.005782  0.009343  pass
32             interoceptive_lancaster      712  0.005345  0.003233  pass
57  distance_semantic_neighbors_shaoul      881  0.003011  0.008346  pass
35              mouth_throat_lancaster      712  0.001949  0.008341  pass
29                    haptic_lancaster      712  0.001466  0.013712  pass
36                  hand_arm_lancaster      712 -0.000632  0.015734  pass
68    visual_lexical_accuracy_keuleers      793 -0.001959  0.002592  pass


,embed,embed_type,norm,train_n,test_n,p,r2_mean,r2_sd,check
0,fMRI_text_cognival_participant,None,frequency_lund,905,227,1000,-0.028576,0.029151,pass
1,fMRI_text_cognival_participant,None,frequency_kucera,872,219,1000,-0.049635,0.057808,pass
2,fMRI_text_cognival_participant,None,frequency_subtlexus,877,220,1000,-0.046786,0.077060,pass
3,fMRI_text_cognival_participant,None,frequency_subtlexuk,880,221,1000,-0.037376,0.066358,pass
4,fMRI_text_cognival_participant,None,frequency_blog_gimenes,884,222,1000,-0.026952,0.036523,pass
...,...,...,...,...,...,...,...,...,...
699,fMRI_text_denoise_512d,None,iconicity_winter2023,599,150,512,-0.035473,0.027085,pass
700,fMRI_text_denoise_512d,None,aoa_schock,126,32,512,-0.027804,0.034623,pass
701,fMRI_text_denoise_512d,None,aoa_rt_schock,126,32,512,-0.041706,0.036968,pass
702,fMRI_text_denoise_512d,None,iconicity_winter2017,320,80,512,-0.068191,0.064661,pass


## Plotting

In [28]:
# Adding norm category
results['norm_category'] = (
    results['norm']
    .apply(lambda norm: norms_meta.loc[norm]['category'])
    .replace({'_': ' '}, regex=True)
)

results_avg = (
    results[['norm_category', 'embed', 'r2_mean']]
    .groupby(['norm_category', 'embed'], as_index=False).median()
    .dropna()
)

results_avg_piv = results_avg.pivot(columns='embed', index='norm_category', values='r2_mean')
results_avg_piv.round(2)

embed,fMRI_text_cognival,fMRI_text_cognival_participant,fMRI_text_denoise_128d,fMRI_text_denoise_128d_participant,fMRI_text_denoise_256d,fMRI_text_denoise_256d_participant,fMRI_text_denoise_512d,fMRI_text_denoise_512d_participant
norm_category,,,,,,,,
age of acquisition,-0.02,-0.04,-0.03,-0.03,-0.03,-0.03,-0.03,-0.03
arousal,-0.02,-0.03,-0.02,-0.03,-0.02,-0.02,-0.01,-0.02
auditory lexical decision,-0.03,-0.03,-0.03,-0.03,-0.03,-0.03,-0.03,-0.03
concreteness,-0.01,-0.02,-0.01,-0.02,-0.00,-0.02,0.01,-0.01
dominance,0.00,-0.02,-0.02,-0.01,-0.03,-0.02,-0.05,-0.02
familiarity,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01
frequency,0.08,-0.04,-0.03,-0.04,-0.04,-0.04,-0.02,-0.02
iconicity/transparency,-0.03,-0.06,-0.05,-0.06,-0.05,-0.06,-0.05,-0.06
imageability,-0.00,-0.05,-0.02,-0.02,0.01,-0.02,-0.00,-0.02


In [ ]:
# Plotting

# Saving

In [ ]:
# Subsetting to only the words in norms
to_pull = set(
    pd.read_csv('../../data/psychNorms/psychNorms.zip', index_col=0, low_memory=False, compression='zip').index
)
fMRI_text_cognival = fMRI_text_cognival.loc[fMRI_text_cognival.index.isin(to_pull)].astype(float)
fMRI_speech_cognival = fMRI_speech_cognival.loc[fMRI_speech_cognival.index.isin(to_pull)].astype(float)

# Saving 
fMRI_text_cognival.to_csv('../../data/embeds/fMRI_text_cognival.csv')
fMRI_speech_cognival.to_csv('../../data/embeds/fMRI_speech_cognival.csv')